In [38]:
import pandas as pd
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from  sklearn.manifold import TSNE
import os
import tensorflow as tf
import numpy as np
import re
from tensorflow.contrib.tensorboard.plugins import projector

In [22]:
with open('../data/txt/texto_limpio.txt', 'r') as f:
    text =f.readlines()
    f.close() 

In [2]:
model = FastText.load_fasttext_format('fasttext_word_vectors_embeddings.bin')

In [41]:
text= [re.sub('\d+', '<NUM>', t) for t in text]

In [42]:
cv= CountVectorizer(stop_words=stopwords.words('spanish'))

vectorized_text = cv.fit_transform(text)

sum_words = vectorized_text.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

top= list(map(lambda x: x[0],words_freq))[0:10000]



In [43]:
embedding = np.array(list(map(lambda x: model.wv[x], top)))

In [45]:


# setup a TensorFlow session
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.Variable([0.0], name='embedding')
place = tf.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.assign(X, place, validate_shape=False)
sess.run(tf.global_variables_initializer())
sess.run(set_x, feed_dict={place: embedding})

# write labels
with open('log/metadata.tsv', 'w') as f:
    for word in top:
        f.write(word + '\n')

# create a TensorFlow summary writer
summary_writer = tf.summary.FileWriter('log', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = 'embedding:0'
embedding_conf.metadata_path = os.path.join('metadata.tsv')
projector.visualize_embeddings(summary_writer, config)

# save the model
saver = tf.train.Saver()
saver.save(sess, os.path.join('log', "model.ckpt"))

/home/dkozlows/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


'log/model.ckpt'